In [1]:
import json
import time
from arango import ArangoClient
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler


class MyHandler(FileSystemEventHandler):
    def on_modified(self, event):
        if event.src_path.endswith('.json'):
            update_arango_db(event.src_path)

def update_arango_db(json_file_path):
    file_name = json_file_path.split('/')[-1].split('.')[0]

    if file_name in ['Movie', 'Theater', 'ShowTime', 'Reservation', 'Customer']:
        update_collection(json_file_path)
        print(f"Collection updated: {file_name}")

def update_collection(json_file_path):
    collection_name = json_file_path.split('/')[-1].split('.')[0]

    # Connect to the ArangoDB server
    client = ArangoClient(hosts='http://localhost:8529')
    db = client.db('_system', username='root', password='')

    # Clear the existing collection
    if db.has_collection(collection_name):
        collection = db.collection(collection_name)
        collection.truncate()
    else:
        # Create the collection if it doesn't exist
        db.create_collection(collection_name)
        collection = db.collection(collection_name)

    # Import data from the JSON file
    with open(json_file_path, 'r') as f:
        data = json.load(f)
        for document in data:
            collection.insert(document)

def main():
    path = "/Users/tanujverma/Desktop/NEU/ADBMS/ArangoDB"  # Set your path to the JSON files
    event_handler = MyHandler()
    observer = Observer()
    observer.schedule(event_handler, path, recursive=False)
    observer.start()

    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        observer.stop()

    observer.join()

if __name__ == "__main__":
    main()



Collection updated: Customer
